In [1]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.onnx

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784,500)
        self.fc2 = nn.Linear(500, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x))
        return x

#モデル定義
model = Net()
x = Variable(torch.randn(1, 784))
torch.onnx.export(model, x, 'model.onnx', verbose=True)

graph(%0 : Float(1, 784)
      %1 : Float(500, 784)
      %2 : Float(500)
      %3 : Float(84, 500)
      %4 : Float(84)
      %5 : Float(10, 84)
      %6 : Float(10)) {
  %7 : Float(1, 500) = onnx::Gemm[alpha=1, beta=1, broadcast=1, transB=1](%0, %1, %2), scope: Net/Linear[fc1]
  %8 : Float(1, 500) = onnx::Relu(%7), scope: Net
  %9 : Float(1, 84) = onnx::Gemm[alpha=1, beta=1, broadcast=1, transB=1](%8, %3, %4), scope: Net/Linear[fc2]
  %10 : Float(1, 84) = onnx::Relu(%9), scope: Net
  %11 : Float(1, 10) = onnx::Gemm[alpha=1, beta=1, broadcast=1, transB=1](%10, %5, %6), scope: Net/Linear[fc3]
  %12 : Float(1, 10) = onnx::LogSoftmax[axis=1](%11), scope: Net
  return (%12);
}



/mnt/f/Linux_home/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [ ]:
import numpy as np
import onnx
from onnx_caffe2.backend import prepare

In [ ]:
model = onnx.load("model.onnx")
print(onnx.helper.printable_graph(model.graph))

In [ ]:
model = onnx.load("model.onnx")
rep = prepare(model)
outputs = rep.run(np.random.randn(1, 1, 28, 28).astype(np.float32))
print(outputs[0])

In [ ]:
# モデルの入力データ一覧を出力する
print("====== Inputs ======")
for i, input in enumerate(model.graph.input):
    print("[Input #{}]".format(i))
    print(input)

In [ ]:
# モデルの出力データ一覧を出力する
print("====== Outputs ======")
for i, output in enumerate(model.graph.output):
    print("[Output #{}]".format(i))
    print(output)

In [1]:
from caffe2.python import core

In [ ]:
import onnx_caffe2.backend